In [9]:
## Built of Demo codes for "Deep Networks Always Grok and Here is Why", ArXiv 2024
## Authors: Ahmed Imtiaz Humayun, Randall Balestriero, Richard Baraniuk

#@title License
#
#The MIT License (MIT)
#Copyright © 2024 Ahmed Imtiaz Humayun
#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
#

#@title Setup

%cd /content/
!git clone https://github.com/AhmedImtiazPrio/grok-adversarial.git
%cd 'grok-adversarial'

!pip install ml_collections
!pip install wandb
!apt update && apt install -y --no-install-recommends libopencv-dev libturbojpeg-dev
!cp -f /usr/lib/x86_64-linux-gnu/pkgconfig/opencv.pc /usr/lib/x86_64-linux-gnu/pkgconfig/opencv4.pc
!pip install ffcv numba opencv-python
!pip install einops

In [10]:
#@title Imports
%cd 'grok-adversarial'

import torch as ch
import torchvision
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, lr_scheduler, AdamW
import numpy as np

import ml_collections
from tqdm import tqdm
import os
import time
import logging

import wandb


from dataloaders import cifar10_dataloaders, cifar10_dataloaders_ffcv, get_LC_samples
from models import make_resnet18k
from utils import flatten_model, add_hooks_preact_resnet18
from attacks import PGD
from local_complexity import get_intersections_for_hulls
from samplers import get_ortho_hull_around_samples, get_ortho_hull_around_samples_w_orig

[Errno 2] No such file or directory: 'grok-adversarial'
/home/niket/grokking/grok-adversarial


/home/niket/miniconda3/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [11]:
#@title Train and evaluation functions

def train(model, loaders, config, add_hook_fn, hulls=None):

    model.cuda()

    ## setup optimizer
    if config.optimizer == 'sgd':
        print('Using SGD optimizer')
        opt = SGD(model.parameters(),
                  lr=config.lr,
                  momentum=config.momentum,
                  weight_decay=config.weight_decay)

    elif config.optimizer == 'adam':
        opt = AdamW(model.parameters(),
                    lr=config.lr,
                    weight_decay=config.weight_decay)

    else:
        raise NotImplementedError

    ## resume training
    if config.resume_step>0 and config.resume_dir is not None:

        assert os.path.exists(
            os.path.join(
                config.load_dir, f'checkpoint-s:{config.resume_step}.pt'
                )
            ), f"Resume checkpoint not found"

        base_chkpt = ch.load(os.path.join(config.resume_dir,
                             f'checkpoint-s:{-1}.pt'))
        model = base_chkpt['model']
        opt = base_chkpt['optimizer']
        state_chkpt = ch.load(os.path.join(config.resume_dir,
                              f'checkpoint-s:{config.resume_step}.pt'))
        model.load_state_dict(state_chkpt['model_state_dict'])
        opt.load_state_dict(state_chkpt['optimizer_state_dict'])


    ### save model and optimizer before training
    ### shortcut to avoid removing hooks later
    ch.save(
              {
                  'model': model,
                  'optimizer': opt,
              },
              os.path.join(
                  config.model_dir,
                  f'checkpoint-s:{-1}.pt'
              )
    )

    iters_per_epoch = len(loaders['train'])
    epochs = np.floor(config.num_steps/iters_per_epoch)

    if config.lr_schedule_flag:
        # Cyclic LR with single triangle
        print('Using Learning Rate Schedule')
        lr_schedule = np.interp(np.arange((epochs+1) * iters_per_epoch),
                                [0, config.lr_peak_epoch * iters_per_epoch, epochs * iters_per_epoch],
                                [0, 1, 0])

        scheduler = lr_scheduler.LambdaLR(opt, lr_schedule.__getitem__)

    loss_fn = CrossEntropyLoss(
        label_smoothing=config.label_smoothing
        )


    train_step = 0 if config.resume_step <= 0 else config.resume_step

    ## stat dict for plotting convenience
    stat_names = ['train_acc','train_loss','test_loss',
                  'test_acc','adv_acc', 'train_step', 'l2', 'last_layer_l2', 'first_layer_dist_to_init', 'second_layer_dist_to_init'] + \
            [each+'_LC' for each in list(hulls.keys())]

    stats = dict(zip(stat_names,[[] for _ in stat_names]))

    print(f'Logging stats for steps:{config.log_steps}')

    while True:

        if train_step > config.num_steps: break

        for ims, labs in tqdm(loaders['train'], desc=f"train_step:{train_step}-{train_step+iters_per_epoch}"):

            ### train step
            ims = ims.cuda()
            labs = ch.eye(10)[labs].float().cuda()

            opt.zero_grad()
            out = model(ims)

            loss = loss_fn(out, labs)
            loss.backward()
            opt.step()
            train_step += 1

            if config.lr_schedule_flag:
                scheduler.step()

            ### log step
            if train_step in config.log_steps:
                print('Computing stats...')

                model.eval()

                ### checkpoint before anything
                if config.save_model:

                  ch.save(
                      {
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': opt.state_dict(),
                      },
                      os.path.join(
                          config.model_dir,
                          f'checkpoint-s:{train_step}.pt'
                      )
                  )

                ## evaluate on train and test
                train_acc, train_loss = evaluate(model,
                                                 loaders['train'],
                                                 loss_fn)
                test_acc, test_loss = evaluate(model,
                                                 loaders['test'],
                                                 loss_fn)
                l2_norm_in = ch.linalg.norm(model.input_layer.weight).item()
                l2_last_layer_norm = ch.linalg.norm(model.output_layer.weight).item()
                first_layer_dist_to_init, second_layer_dist_to_init = model.get_dist_to_init()

                stats['train_acc'].append(train_acc)
                stats['test_acc'].append(test_acc)
                stats['train_loss'].append(train_loss)
                stats['test_loss'].append(test_loss)
                stats['train_step'].append(train_step)
                stats['l2'].append(l2_norm_in)
                stats['last_layer_l2'].append(l2_last_layer_norm)
                stats['first_layer_dist_to_init'].append(first_layer_dist_to_init)
                stats['second_layer_dist_to_init'].append(second_layer_dist_to_init)

                ## evaluate local complexity
                # add hooks
                if config.compute_LC:

                  model, layer_names, activation_buffer = add_hook_fn(model, config)

                  if hulls is not None:
                    for k in hulls.keys():

                      # compute number of neurons that intersect hulls
                      # using network activations

                      with ch.no_grad():

                        n_inters, _ = get_intersections_for_hulls(
                                        hulls[k],
                                        model=model,
                                        batch_size=config.LC_batch_size,
                                        layer_names=layer_names,
                                        activation_buffer=activation_buffer
                                  )

                      stats[k+'_LC'].append(n_inters.cpu())

                ## evaluate robustness
                if config.compute_robust:

                  adv_acc = evaluate_adv(model, loaders['test'], config)
                  stats['adv_acc'].append(adv_acc)


                if config.wandb_log:
                  wandb.log({
                      'iter' : train_step,
                      'train/acc': stats['train_acc'][-1],
                      'train/loss': stats['train_loss'][-1],
                      'test/acc' : stats['test_acc'][-1],
                      'test/loss' : stats['test_loss'][-1],
                      'train/LC' : stats['train_LC'][-1].sum(1).mean(0),
                      'test/LC' : stats['test_LC'][-1].sum(1).mean(0),
                      'random/LC' : stats['rand_LC'][-1].sum(1).mean(0),
                      'adv/acc' : stats['adv_acc'][-1],
                      'l2' : stats['l2'][-1],
                      'last_layer_l2' : stats['last_layer_l2'][-1],
                      'first_layer_dist_to_init' : stats['first_layer_dist_to_init'][-1],
                      'second_layer_dist_to_init' : stats['second_layer_dist_to_init'][-1]
                  })

                model.train()


    ## save after training is complete
    ch.save(
        {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
        },
        os.path.join(
            config.model_dir,
            f'checkpoint-s:{train_step}.pt'
        )
    )

    return stats

@ch.no_grad
def evaluate(model, dloader, loss_fn=None):
  
  model.eval()

  acc = 0
  loss = 0
  nsamples = 0
  nbatch = 0
  
  print(f"Evaluating on {len(dloader)} batches")

  for ims, labs in dloader:
      print(f"eval batch norm is: {ch.norm(ims)}")

      ims = ims.cuda()
      labs = labs.cuda()
      outs = model(ims)

      if loss_fn is not None:
        loss += loss_fn(outs, labs).item()
        nbatch += 1
        
      batch_acc = ch.sum(labs == outs.argmax(dim=-1)).item() / outs.shape[0]

      print(f"Batch acc: {batch_acc}")
      
      acc += ch.sum(labs == outs.argmax(dim=-1)).cpu()
      nsamples += outs.shape[0]

  return acc/nsamples, loss/nbatch

def evaluate_adv(model, dloader, config):

  atk = PGD(model,
          eps=config.atk_eps,
          alpha=config.atk_alpha,
          steps=config.atk_itrs,
          dmin=config.dmin,
          dmax=config.dmax
          )

  acc = 0
  nsamples = 0
  for ims, labs in tqdm(dloader, desc=f"Computing robust acc for eps:{config.atk_eps:.3f}"):

    ims = ims.cuda()
    labs = labs.cuda()

    adv_images = atk(ims, labs)

    with ch.no_grad():
        adv_pred = model(adv_images).argmax(dim=-1)

    acc += ch.sum(labs == adv_pred).cpu()
    nsamples += adv_pred.shape[0]

  return acc/nsamples

In [12]:
def add_hooks_MLP(model, config, verbose=False):
    """
    Add hooks to preact resnet
    """

    names,modules = flatten_model(model)
    assert len(names) == len(modules)

    ## add hooks to linear layers
    layer_ids = np.asarray([i for i,each in enumerate(modules) if (type(each)==ch.nn.Linear)])

    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    for each in layer_ids:
        modules[each].register_forward_hook(get_activation(names[each]))

    layer_names = np.sort(np.asarray(names)[layer_ids])

    if verbose:
        print('Adding Hook to',layer_names)

    return model, layer_names, activation


In [13]:
def get_config():
  """hyperparameter configuration."""
  config = ml_collections.ConfigDict()

  config.optimizer = 'sgd'
  config.lr = 1e-2
  config.momentum = 0.00
  config.lr_schedule_flag = False

  config.train_batch_size = 64
  config.test_batch_size = 64
  config.num_steps = 500000                       # number of training steps
  config.weight_decay = 0.001
  config.label_smoothing = 0.0
  config.log_steps = np.unique(
      np.logspace(0,5.7,50).astype(int).clip(
          0,config.num_steps
          )
      )
  config.seed = 42

  config.dmax = 1
  config.dmin = 0

  config.save_model = False                      # save every model checkpoint
  config.wandb_log = True                        # log using wandb
  config.save_model = False                      # save every model checkpoint
  config.wandb_log = True                        # log using wandb
  config.wandb_proj = 'grok-adv-MLP-MNIST'
  config.wandb_pref = 'MNIST-MLP'

  config.resume_step = 0

  ## mlp params
  config.input_dim = 784
  config.output_dim = 10
  config.hidden_dim = 1024
  config.input_weights_init_scale = 8 #sets initialization standard deviation to be = (input_weights_init_scale)/sqrt(input_dim) 
  config.output_weights_init_scale = 8


  ## local complexity approx. parameters
  config.compute_LC = True
  config.approx_n = 1024                         # number of samples to use for approximation
  config.n_frame = 40                            # number of vertices for neighborhood
  config.r_frame = 0.005                         # radius of \ell-1 ball neighborhood
  config.LC_batch_size = 256
  config.inc_centroid = False                     # include original sample as neighborhood vertex


  ## adv robustness parameters
  config.compute_robust = True                   # note that if normalize==True, data is not bounded between [0,1]
  config.atk_eps = 128/255   ## 8/255
  config.atk_alpha = 10/255  ## 2/255
  config.atk_itrs = 10

  return config


config = get_config()

In [14]:
## load data
train_loader = ch.utils.data.DataLoader(torchvision.datasets.MNIST('../mnist_data',
                                                          download=True,
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])),
                                           batch_size=config.train_batch_size,
                                           shuffle=True)

test_loader = ch.utils.data.DataLoader(torchvision.datasets.MNIST('../mnist_data',
                                                          download=True,
                                                          train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs,
                                                          ])),
                                           batch_size=config.test_batch_size,
                                           shuffle=True)


## initialize neighborhood sampler
sampler_params = {'n' : config.n_frame if not config.inc_centroid \
                    else config.n_frame+1,
                  'r' : config.r_frame, 'seed':config.seed}

sampler = get_ortho_hull_around_samples_w_orig if config.inc_centroid \
            else get_ortho_hull_around_samples

## select samples for neighborhood computation
train_LC_batch, _ = get_LC_samples(train_loader,config)
test_LC_batch, _ = get_LC_samples(test_loader,config)
rand_LC_batch = ch.randn_like(train_LC_batch)

## sample hulls/neighborhoods
train_hulls = sampler(
  train_LC_batch.cuda(),
  **sampler_params
    ).cpu()
test_hulls = sampler(
    test_LC_batch.cuda(),
    **sampler_params
).cpu()
rand_hulls = sampler(
    rand_LC_batch.cuda(),
    **sampler_params
).cpu()


## make hull dict. the keys of this dict will be used for logging
## you can add hulls separately for different classes as well to
## keep track of classwise statistics
hulls = {
    'train' : train_hulls,
    'test' : test_hulls,
    'rand' : rand_hulls
}

In [15]:
loaders = {
    'train' : train_loader,
    'test' : test_loader
}

## directory for saving logs and models
timestamp = time.ctime().replace(' ','_')
config.model_dir = os.path.join(f'./models/{timestamp}')
config.log_dir = os.path.join(f'./logs/{timestamp}')
os.mkdir(config.model_dir)
os.mkdir(config.log_dir)

In [16]:
if config.wandb_log:
  wandb_project = config.wandb_proj
  wandb_run_name = f"{config.wandb_pref}-{timestamp}"
  wandb.init(project=wandb_project, name=wandb_run_name, config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: niketpatel (niket). Use `wandb login --relogin` to force relogin


In [17]:
from torch import nn
import einops

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, input_std_init = 1, output_std_init = 1):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Define input layer
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.initial_init = ch.normal(mean = ch.zeros(hidden_dim, input_dim), std = input_std_init)
        self.input_layer.weight = nn.parameter.Parameter(self.initial_init.clone())
        self.input_activation = nn.ReLU()

        # Define output layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.output_init = ch.normal(mean = ch.zeros(output_dim, hidden_dim), std = output_std_init)
        self.output_layer.weight = nn.parameter.Parameter(self.output_init.clone())
        self.output_activation = nn.Softmax()

    def forward(self, x):
        x = einops.rearrange(x, "b c u i -> b c (u i)")
        x = self.input_activation(self.input_layer(x))
        x = self.output_activation(self.output_layer(x))
        x = einops.rearrange(x, "x 1 n -> x n")
        return x.float()

    def get_dist_to_init(self):
        layer_1_dif = ch.norm(self.input_layer.weight.detach().cpu() - self.initial_init.detach().cpu()).detach().item()
        layer_2_dif = ch.norm(self.output_layer.weight.detach().cpu() - self.output_init.detach().cpu()).detach().item()
        return layer_1_dif, layer_2_dif

In [18]:
## create model
model = MLP(input_dim = config.input_dim, hidden_dim = config.hidden_dim, output_dim = config.output_dim, input_std_init=(config.input_weights_init_scale/np.sqrt(config.input_dim)), output_std_init=(config.output_weights_init_scale/np.sqrt(config.hidden_dim)))

stats = train(model, loaders,
      config=config, hulls=hulls,
      add_hook_fn=add_hooks_MLP
      )

Using SGD optimizer
Logging stats for steps:[     1      2      3      4      6      8     11     14     19     24
     32     42     55     72     94    124    162    212    277    362
    473    619    809   1058   1382   1807   2363   3088   4037   5277
   6898   9017  11787  15408  20141  26327  34413  44984  58801  76862
 100471 131331 171669 224398 293323 383418 500000]


train_step:0-938:   0%|          | 0/938 [00:00<?, ?it/s]

Computing stats...
Evaluating on 938 batches
eval batch norm is: 226.72068786621094
Batch acc: 0.046875
eval batch norm is: 229.74172973632812
Batch acc: 0.109375
eval batch norm is: 223.88421630859375
Batch acc: 0.109375
eval batch norm is: 230.67649841308594
Batch acc: 0.140625
eval batch norm is: 219.66876220703125
Batch acc: 0.15625
eval batch norm is: 224.59732055664062
Batch acc: 0.109375
eval batch norm is: 226.1856231689453
Batch acc: 0.203125
eval batch norm is: 224.95492553710938
Batch acc: 0.1875
eval batch norm is: 218.2235565185547
Batch acc: 0.15625
eval batch norm is: 229.99786376953125
Batch acc: 0.109375
eval batch norm is: 221.5867156982422
Batch acc: 0.140625
eval batch norm is: 225.6341552734375
Batch acc: 0.15625
eval batch norm is: 222.9871826171875
Batch acc: 0.21875
eval batch norm is: 226.48211669921875
Batch acc: 0.140625
eval batch norm is: 228.61231994628906
Batch acc: 0.171875
eval batch norm is: 230.17660522460938
Batch acc: 0.109375
eval batch norm is: 22

/home/niket/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


eval batch norm is: 224.13194274902344
Batch acc: 0.1875
eval batch norm is: 226.90426635742188
Batch acc: 0.0625
eval batch norm is: 225.77528381347656
Batch acc: 0.15625
eval batch norm is: 220.52882385253906
Batch acc: 0.203125
eval batch norm is: 220.76431274414062
Batch acc: 0.09375
eval batch norm is: 215.82846069335938
Batch acc: 0.140625
eval batch norm is: 225.37899780273438
Batch acc: 0.09375
eval batch norm is: 222.4685516357422
Batch acc: 0.203125
eval batch norm is: 214.99246215820312
Batch acc: 0.078125
eval batch norm is: 223.06126403808594
Batch acc: 0.09375
eval batch norm is: 222.23580932617188
Batch acc: 0.125
eval batch norm is: 228.73382568359375
Batch acc: 0.21875
eval batch norm is: 229.8024444580078
Batch acc: 0.078125
eval batch norm is: 227.27627563476562
Batch acc: 0.125
eval batch norm is: 222.1641082763672
Batch acc: 0.1875
eval batch norm is: 211.33758544921875
Batch acc: 0.21875
eval batch norm is: 225.4578857421875
Batch acc: 0.15625
eval batch norm is: 

...

train_step:75978-76916:  93%|█████████▎| 870/938 [00:05<00:00, 136.09it/s]

Computing stats...
Evaluating on 938 batches
eval batch norm is: 223.7159881591797
Batch acc: 0.703125
eval batch norm is: 220.3102569580078
Batch acc: 0.6875
eval batch norm is: 218.9637908935547
Batch acc: 0.671875
eval batch norm is: 229.91366577148438
Batch acc: 0.828125
eval batch norm is: 219.10256958007812
Batch acc: 0.578125
eval batch norm is: 222.505615234375
Batch acc: 0.84375
eval batch norm is: 225.50289916992188
Batch acc: 0.78125
eval batch norm is: 219.49342346191406
Batch acc: 0.703125
eval batch norm is: 224.0406494140625
Batch acc: 0.734375
eval batch norm is: 222.4488067626953
Batch acc: 0.5625
eval batch norm is: 221.3625946044922
Batch acc: 0.734375
eval batch norm is: 225.5829620361328
Batch acc: 0.84375
eval batch norm is: 232.92709350585938
Batch acc: 0.6875
eval batch norm is: 218.14688110351562
Batch acc: 0.703125
eval batch norm is: 227.07752990722656
Batch acc: 0.765625
eval batch norm is: 226.7881317138672
Batch acc: 0.71875
eval batch norm is: 225.7957611

train_step:100366-101304:  10%|▉         | 90/938 [00:00<00:05, 147.08it/s]

Computing stats...
Evaluating on 938 batches
eval batch norm is: 228.81967163085938
Batch acc: 0.8125
eval batch norm is: 220.3187713623047
Batch acc: 0.765625
eval batch norm is: 220.2303009033203
Batch acc: 0.625
eval batch norm is: 215.74749755859375
Batch acc: 0.796875
eval batch norm is: 218.9896697998047
Batch acc: 0.671875
eval batch norm is: 222.07208251953125
Batch acc: 0.765625
eval batch norm is: 224.34559631347656
Batch acc: 0.796875
eval batch norm is: 223.50653076171875
Batch acc: 0.65625
eval batch norm is: 218.53041076660156
Batch acc: 0.6875
eval batch norm is: 221.2300262451172
Batch acc: 0.71875
eval batch norm is: 217.7039031982422
Batch acc: 0.765625
eval batch norm is: 226.40093994140625
Batch acc: 0.84375
eval batch norm is: 215.2310028076172
Batch acc: 0.734375
eval batch norm is: 216.8800048828125
Batch acc: 0.796875
eval batch norm is: 230.54080200195312
Batch acc: 0.765625
eval batch norm is: 221.26361083984375
Batch acc: 0.78125
eval batch norm is: 220.00701

train_step:111622-112560:  92%|█████████▏| 861/938 [00:05<00:00, 150.90it/s]